# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA5D_10/NACA5D_10.csv")

In [4]:
# printing the first 10 rows of the dataset
df.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.1,100000,-1.140,0.02494,0.005
1,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.1,100000,-1.027,0.02116,0.004
2,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.1,100000,-0.914,0.01697,0.004
3,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.1,100000,-0.800,0.01364,0.003
4,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.1,100000,-0.687,0.02483,0.003
5,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.1,100000,-0.572,0.01565,0.002
6,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.1,100000,-0.458,0.01114,0.002
7,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.1,100000,-0.344,0.00882,0.001
8,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.1,100000,-0.229,0.01391,0.001
9,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.1,100000,-0.115,0.01357,0.000


In [5]:
# checking for missing values in the dataset
df.isnull().sum()

t            0
CL_design    0
p            0
s            0
yU1          0
yU2          0
yU3          0
yU4          0
yU5          0
yU6          0
yU7          0
yU8          0
yU9          0
yU10         0
yL1          0
yL2          0
yL3          0
yL4          0
yL5          0
yL6          0
yL7          0
yL8          0
yL9          0
yL10         0
alpha        0
M            0
Re           0
CL           0
CD           0
Cm           0
dtype: int64

In [6]:
# description of the dataset
df.describe()

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,...,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000
mean,19.964508,0.841171,27.648625,0.493345,0.061993,0.102135,0.127800,0.134156,0.122524,0.098694,...,-0.050760,-0.035252,-0.019158,-0.006666,0.000000,0.200000,300000.000000,0.219104,0.025913,0.008665
std,10.002176,0.595675,16.859596,0.499957,0.035497,0.050154,0.062040,0.067278,0.064190,0.054114,...,0.038246,0.024719,0.012716,0.004107,6.055314,0.081686,141421.655012,0.926011,0.011283,0.092562
min,5.000000,0.000000,5.000000,0.000000,0.002846,-0.006454,-0.022993,-0.039093,-0.048037,-0.046811,...,-0.171128,-0.127512,-0.072044,-0.023250,-10.000000,0.100000,100000.000000,-2.607000,0.002690,-0.169000
25%,10.000000,0.200000,15.000000,0.000000,0.033635,0.064809,0.081379,0.082846,0.074521,0.059894,...,-0.078123,-0.052613,-0.027989,-0.009654,-5.000000,0.100000,200000.000000,-0.485000,0.018260,-0.031000
50%,20.000000,0.800000,25.000000,0.000000,0.057851,0.101474,0.127721,0.133115,0.120174,0.099175,...,-0.050658,-0.034769,-0.018871,-0.006613,-0.000000,0.200000,300000.000000,0.218000,0.024650,-0.004000
75%,30.000000,1.400000,45.000000,1.000000,0.085249,0.134294,0.169672,0.179965,0.165684,0.137987,...,-0.022239,-0.016698,-0.009816,-0.003398,5.000000,0.300000,400000.000000,0.928000,0.031580,0.018000
max,35.000000,1.800000,75.000000,1.000000,0.165549,0.250220,0.311403,0.334353,0.308679,0.262679,...,0.048283,0.022775,0.009953,0.002097,10.000000,0.300000,500000.000000,3.020000,0.617020,0.453000


In [7]:
# shuffling all the rows
df = df.sample(frac=1)
df.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
9816,20,0.0,25,0,0.039300,0.071524,0.092396,0.099961,0.094677,0.079859,...,-0.059794,-0.038557,-0.019793,-0.006775,-1.0,0.1,300000,-0.129,0.01745,0.002
157913,10,1.2,25,0,0.044387,0.092831,0.128847,0.136949,0.119785,0.094543,...,0.006876,0.002465,0.000118,-0.000825,4.0,0.3,500000,1.261,0.02090,-0.094
116375,15,0.8,25,1,0.054190,0.101469,0.136266,0.145355,0.126765,0.094545,...,-0.029250,-0.024358,-0.014778,-0.005432,4.0,0.3,200000,0.880,0.02432,0.008
117647,35,0.8,25,1,0.121125,0.183771,0.229385,0.245217,0.226751,0.181142,...,-0.087588,-0.062450,-0.034711,-0.012339,-5.0,0.3,300000,-0.257,0.03177,0.041
227438,30,1.8,25,1,0.147247,0.229640,0.288999,0.309148,0.281126,0.213573,...,-0.052789,-0.046796,-0.029641,-0.011138,-2.0,0.3,100000,0.712,0.03973,0.033
209024,20,1.6,35,1,0.078698,0.141250,0.193904,0.221937,0.216856,0.179758,...,-0.002563,-0.014976,-0.015025,-0.006861,1.0,0.1,400000,1.030,0.02821,-0.091
66569,10,0.4,15,1,0.031307,0.059290,0.073097,0.071998,0.063459,0.050366,...,-0.024202,-0.016726,-0.009050,-0.003227,10.0,0.1,500000,1.329,0.02265,-0.004
130393,30,1.0,5,1,0.108273,0.144520,0.164482,0.162304,0.135309,0.095215,...,-0.126813,-0.090530,-0.049358,-0.016031,-6.0,0.3,500000,-1.476,0.01178,0.262
126029,30,0.8,45,1,0.061675,0.110384,0.142699,0.154975,0.147623,0.125228,...,-0.085133,-0.054541,-0.027626,-0.008961,-2.0,0.1,200000,-0.207,0.02800,-0.009
218220,25,1.8,5,1,0.143118,0.172861,0.173581,0.150006,0.104282,0.052359,...,-0.139515,-0.104860,-0.058821,-0.018597,-1.0,0.2,200000,-1.153,0.02937,0.426


In [8]:
# dropping columns FoilID, m, p & t
df = df.drop(columns=['t', 'CL_design', 'p', 's'], axis=1)
df.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
9816,0.039300,0.071524,0.092396,0.099961,0.094677,0.079859,0.059794,0.038557,0.019793,0.006775,...,-0.059794,-0.038557,-0.019793,-0.006775,-1.0,0.1,300000,-0.129,0.01745,0.002
157913,0.044387,0.092831,0.128847,0.136949,0.119785,0.094543,0.067224,0.041374,0.020077,0.005984,...,0.006876,0.002465,0.000118,-0.000825,4.0,0.3,500000,1.261,0.02090,-0.094
116375,0.054190,0.101469,0.136266,0.145355,0.126765,0.094545,0.061297,0.033778,0.014999,0.004743,...,-0.029250,-0.024358,-0.014778,-0.005432,4.0,0.3,200000,0.880,0.02432,0.008
117647,0.121125,0.183771,0.229385,0.245217,0.226751,0.181142,0.126091,0.074771,0.035490,0.011568,...,-0.087588,-0.062450,-0.034711,-0.012339,-5.0,0.3,300000,-0.257,0.03177,0.041
227438,0.147247,0.229640,0.288999,0.309148,0.281126,0.213573,0.137436,0.073374,0.031151,0.009453,...,-0.052789,-0.046796,-0.029641,-0.011138,-2.0,0.3,100000,0.712,0.03973,0.033
209024,0.078698,0.141250,0.193904,0.221937,0.216856,0.179758,0.122953,0.065290,0.025253,0.006733,...,-0.002563,-0.014976,-0.015025,-0.006861,1.0,0.1,400000,1.030,0.02821,-0.091
66569,0.031307,0.059290,0.073097,0.071998,0.063459,0.050366,0.035661,0.021867,0.010758,0.003551,...,-0.024202,-0.016726,-0.009050,-0.003227,10.0,0.1,500000,1.329,0.02265,-0.004
130393,0.108273,0.144520,0.164482,0.162304,0.135309,0.095215,0.056348,0.028083,0.011961,0.004814,...,-0.126813,-0.090530,-0.049358,-0.016031,-6.0,0.3,500000,-1.476,0.01178,0.262
126029,0.061675,0.110384,0.142699,0.154975,0.147623,0.125228,0.094242,0.061117,0.031734,0.011337,...,-0.085133,-0.054541,-0.027626,-0.008961,-2.0,0.1,200000,-0.207,0.02800,-0.009
218220,0.143118,0.172861,0.173581,0.150006,0.104282,0.052359,0.012662,-0.005506,-0.006277,-0.000400,...,-0.139515,-0.104860,-0.058821,-0.018597,-1.0,0.2,200000,-1.153,0.02937,0.426


In [9]:
# defining the feature and target columns
X = df.drop(columns=['CL', 'CD', 'Cm'])
y = df[['CL', 'CD', 'Cm']]

In [10]:
# displaying the features table
X.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL4,yL5,yL6,yL7,yL8,yL9,yL10,alpha,M,Re
9816,0.039300,0.071524,0.092396,0.099961,0.094677,0.079859,0.059794,0.038557,0.019793,0.006775,...,-0.099961,-0.094677,-0.079859,-0.059794,-0.038557,-0.019793,-0.006775,-1.0,0.1,300000
157913,0.044387,0.092831,0.128847,0.136949,0.119785,0.094543,0.067224,0.041374,0.020077,0.005984,...,0.036723,0.024285,0.013952,0.006876,0.002465,0.000118,-0.000825,4.0,0.3,500000
116375,0.054190,0.101469,0.136266,0.145355,0.126765,0.094545,0.061297,0.033778,0.014999,0.004743,...,-0.004878,-0.017044,-0.026746,-0.029250,-0.024358,-0.014778,-0.005432,4.0,0.3,200000
117647,0.121125,0.183771,0.229385,0.245217,0.226751,0.181142,0.126091,0.074771,0.035490,0.011568,...,-0.104448,-0.110043,-0.104499,-0.087588,-0.062450,-0.034711,-0.012339,-5.0,0.3,300000
227438,0.147247,0.229640,0.288999,0.309148,0.281126,0.213573,0.137436,0.073374,0.031151,0.009453,...,0.008999,-0.019506,-0.043404,-0.052789,-0.046796,-0.029641,-0.011138,-2.0,0.3,100000
209024,0.078698,0.141250,0.193904,0.221937,0.216856,0.179758,0.122953,0.065290,0.025253,0.006733,...,0.021162,0.026110,0.014865,-0.002563,-0.014976,-0.015025,-0.006861,1.0,0.1,400000
66569,0.031307,0.059290,0.073097,0.071998,0.063459,0.050366,0.035661,0.021867,0.010758,0.003551,...,-0.028063,-0.031339,-0.029596,-0.024202,-0.016726,-0.009050,-0.003227,10.0,0.1,500000
130393,0.108273,0.144520,0.164482,0.162304,0.135309,0.095215,0.056348,0.028083,0.011961,0.004814,...,-0.140848,-0.155266,-0.149946,-0.126813,-0.090530,-0.049358,-0.016031,-6.0,0.3,500000
126029,0.061675,0.110384,0.142699,0.154975,0.147623,0.125228,0.094242,0.061117,0.031734,0.011337,...,-0.144916,-0.136403,-0.114341,-0.085133,-0.054541,-0.027626,-0.008961,-2.0,0.1,200000
218220,0.143118,0.172861,0.173581,0.150006,0.104282,0.052359,0.012662,-0.005506,-0.006277,-0.000400,...,-0.111365,-0.143164,-0.153212,-0.139515,-0.104860,-0.058821,-0.018597,-1.0,0.2,200000


In [11]:
# displaying the targets table
y.head(10)

,CL,CD,Cm
9816,-0.129,0.01745,0.002
157913,1.261,0.02090,-0.094
116375,0.880,0.02432,0.008
117647,-0.257,0.03177,0.041
227438,0.712,0.03973,0.033
209024,1.030,0.02821,-0.091
66569,1.329,0.02265,-0.004
130393,-1.476,0.01178,0.262
126029,-0.207,0.02800,-0.009
218220,-1.153,0.02937,0.426


## Splitting the Dataset

In [12]:
# splitting dataset into training, validation & testing dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [14]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.699176,-0.723555,-0.727746,-0.716627,-0.676742,-0.619930,-0.552796,-0.482956,-0.426563,-0.439161,...,0.029555,0.083232,0.152884,0.231561,0.310078,0.374268,0.407151,-0.993211,1.224890,-1.415684
1,-0.180146,0.411380,0.271218,-0.145946,-0.411258,-0.592308,-0.729927,-0.838807,-0.934841,-1.118918,...,2.102700,1.879202,1.726051,1.603379,1.499346,1.417044,1.387086,-0.993211,0.000842,0.704949
2,0.264264,0.511103,0.597370,0.661700,0.737201,0.833882,0.946167,1.069051,1.201210,1.432880,...,-1.587230,-1.512064,-1.422699,-1.318355,-1.201744,-1.093182,-1.040405,-1.158492,1.224890,0.704949
3,1.974228,1.889203,1.675177,1.511565,1.451535,1.457462,1.499844,1.557396,1.594548,1.531931,...,0.015171,-0.086141,-0.106102,-0.086656,-0.053872,-0.047099,-0.197204,1.155449,0.000842,-0.001929
4,1.785487,1.324755,1.327858,1.359215,1.391993,1.451514,1.530729,1.619504,1.692670,1.720633,...,-0.863378,-0.930578,-0.927351,-0.883192,-0.822050,-0.779258,-0.885021,-0.827929,0.000842,1.411827


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.528266,-0.489719,-0.427556,-0.350478,-0.270859,-0.190669,-0.107433,-0.020905,0.050870,0.061429,...,-0.352346,-0.260421,-0.171247,-0.081078,0.009479,0.079581,0.079804,0.329042,1.224890,-0.001929
1,1.549821,1.111040,0.893904,0.793945,0.605194,0.344797,0.054849,-0.192277,-0.325494,-0.161825,...,-1.498843,-1.789638,-2.094292,-2.403838,-2.677977,-2.847767,-2.785244,-1.489055,1.224890,0.704949
2,-0.638680,-0.609554,-0.569727,-0.507357,-0.433199,-0.347846,-0.253692,-0.157991,-0.075809,-0.041744,...,-0.511561,-0.429147,-0.336388,-0.234790,-0.132594,-0.049162,-0.025793,-0.166803,0.000842,-0.001929
3,-0.639863,-0.610812,-0.571093,-0.508660,-0.434270,-0.348640,-0.254200,-0.158230,-0.075822,-0.041636,...,-0.512883,-0.430261,-0.337224,-0.235326,-0.132842,-0.049177,-0.025688,-0.001521,1.224890,0.704949
4,-0.383379,-0.380566,-0.378064,-0.352676,-0.300232,-0.227448,-0.142016,-0.053106,0.021256,0.037419,...,-0.354878,-0.292340,-0.211658,-0.119379,-0.026017,0.047391,0.053431,-0.166803,-1.223205,-1.415684


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.704206,-0.832156,-0.841684,-0.797493,-0.731918,-0.653311,-0.566266,-0.479071,-0.409707,-0.418183,...,-0.052065,0.025327,0.116455,0.215773,0.312256,0.389512,0.426695,0.990168,1.224890,-0.708806
1,-0.524449,-0.402323,-0.234682,-0.089206,0.013770,0.070141,0.104097,0.146115,0.165485,0.047733,...,0.666113,0.799002,0.869066,0.911538,0.950834,0.963273,0.885965,-1.489055,1.224890,-1.415684
2,1.196811,1.241653,1.298657,1.395383,1.509449,1.606975,1.672193,1.745657,1.804034,1.806965,...,-0.842447,-0.726766,-0.652224,-0.608056,-0.555442,-0.528670,-0.671914,-0.993211,-1.223205,-0.001929
3,0.756546,0.290197,-0.072617,-0.343784,-0.599858,-0.862782,-1.113628,-1.313197,-1.413786,-1.278963,...,-0.389105,-0.655548,-0.920784,-1.175148,-1.379146,-1.474050,-1.319236,1.155449,0.000842,-1.415684
4,1.785487,1.324755,1.327858,1.359215,1.391993,1.451514,1.530729,1.619504,1.692670,1.720633,...,-0.863378,-0.930578,-0.927351,-0.883192,-0.822050,-0.779258,-0.885021,-1.158492,-1.223205,-0.001929


In [15]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
42634,-0.699,0.02283,0.011
221407,-0.118,0.01572,-0.004
104541,-0.987,0.03553,0.008
198719,1.943,0.04321,-0.028
216599,-0.131,0.01922,-0.008


,CL,CD,Cm
49719,0.390,0.02145,-0.021
130684,-1.963,0.04398,0.267
12231,-0.131,0.01934,0.002
10048,0.000,0.01780,-0.000
40644,-0.023,0.02294,-0.003


,CL,CD,Cm
36157,0.836,0.02234,-0.007
140491,-0.601,0.02518,-0.082
141586,-0.174,0.03062,-0.069
129797,0.381,0.01303,0.240
216765,-0.418,0.02261,-0.008


In [16]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

## Network Performance Due to Multiple Hidden Layers

### Case 1: Model [64, 3]

In [17]:
# RMSE and R² metric scores on test data for Model M_64
M_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64...")

    M_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_df = pd.DataFrame(M_64_metrics)

print(results_M_64_df.mean())



Run 1/20 for Model M_64...


I0000 00:00:1764145535.860779 3575415 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764145537.905392 3576544 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_64...

Run 3/20 for Model M_64...

Run 4/20 for Model M_64...

Run 5/20 for Model M_64...

Run 6/20 for Model M_64...

Run 7/20 for Model M_64...

Run 8/20 for Model M_64...

Run 9/20 for Model M_64...

Run 10/20 for Model M_64...

Run 11/20 for Model M_64...

Run 12/20 for Model M_64...

Run 13/20 for Model M_64...

Run 14/20 for Model M_64...

Run 15/20 for Model M_64...

Run 16/20 for Model M_64...

Run 17/20 for Model M_64...

Run 18/20 for Model M_64...

Run 19/20 for Model M_64...

Run 20/20 for Model M_64...
CL_rmse    0.005329
CD_rmse    0.005838
Cm_rmse    0.002324
CL_r2      0.999967
CD_r2      0.721204
Cm_r2      0.999359
dtype: float32


### Case 2: Model [64, 32, 3]

In [18]:
# RMSE and R² metric scores on test data for Model M_64_32
M_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32...")
    
    M_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_df = pd.DataFrame(M_64_32_metrics)

print(results_M_64_32_df.mean())



Run 1/20 for Model M_64_32...

Run 2/20 for Model M_64_32...

Run 3/20 for Model M_64_32...

Run 4/20 for Model M_64_32...

Run 5/20 for Model M_64_32...

Run 6/20 for Model M_64_32...

Run 7/20 for Model M_64_32...

Run 8/20 for Model M_64_32...

Run 9/20 for Model M_64_32...

Run 10/20 for Model M_64_32...

Run 11/20 for Model M_64_32...

Run 12/20 for Model M_64_32...

Run 13/20 for Model M_64_32...

Run 14/20 for Model M_64_32...

Run 15/20 for Model M_64_32...

Run 16/20 for Model M_64_32...

Run 17/20 for Model M_64_32...

Run 18/20 for Model M_64_32...

Run 19/20 for Model M_64_32...

Run 20/20 for Model M_64_32...
CL_rmse    0.004234
CD_rmse    0.005447
Cm_rmse    0.001917
CL_r2      0.999979
CD_r2      0.757878
Cm_r2      0.999566
dtype: float32


### Case 3: Model [64, 32, 16, 3]

In [19]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")
    
    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.004475
CD_rmse    0.005434
Cm_rmse    0.002041
CL_r2      0.999976
CD_r2      0.758979
Cm_r2      0.999510
dtype: float32


### Case 4: Model [64, 32, 16, 8, 3]

In [20]:
# RMSE and R² metric scores on test data for Model M_64_32_16_8
M_64_32_16_8_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16_8...")
    
    M_64_32_16_8 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16_8.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16_8.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16_8.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_8_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_8_df = pd.DataFrame(M_64_32_16_8_metrics)

print(results_M_64_32_16_8_df.mean())



Run 1/20 for Model M_64_32_16_8...

Run 2/20 for Model M_64_32_16_8...

Run 3/20 for Model M_64_32_16_8...

Run 4/20 for Model M_64_32_16_8...

Run 5/20 for Model M_64_32_16_8...

Run 6/20 for Model M_64_32_16_8...

Run 7/20 for Model M_64_32_16_8...

Run 8/20 for Model M_64_32_16_8...

Run 9/20 for Model M_64_32_16_8...

Run 10/20 for Model M_64_32_16_8...

Run 11/20 for Model M_64_32_16_8...

Run 12/20 for Model M_64_32_16_8...

Run 13/20 for Model M_64_32_16_8...

Run 14/20 for Model M_64_32_16_8...

Run 15/20 for Model M_64_32_16_8...

Run 16/20 for Model M_64_32_16_8...

Run 17/20 for Model M_64_32_16_8...

Run 18/20 for Model M_64_32_16_8...

Run 19/20 for Model M_64_32_16_8...

Run 20/20 for Model M_64_32_16_8...
CL_rmse    0.004295
CD_rmse    0.005438
Cm_rmse    0.001958
CL_r2      0.999978
CD_r2      0.758787
Cm_r2      0.999549
dtype: float32


In [21]:
M_64_stats = results_M_64_df.mean()
M_64_32_stats = results_M_64_32_df.mean()
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_64_32_16_8_stats = results_M_64_32_16_8_df.mean()

architectures = [
    '64, 3',
    '64, 32, 3',
    '64, 32, 16, 3',
    '64, 32, 16, 8, 3'
]

table_data = [
    (M_64_stats['CL_rmse'], M_64_stats['CD_rmse'], M_64_stats['Cm_rmse'],
     M_64_stats['CL_r2'], M_64_stats['CD_r2'], M_64_stats['Cm_r2']),
    
    (M_64_32_stats['CL_rmse'], M_64_32_stats['CD_rmse'], M_64_32_stats['Cm_rmse'],
     M_64_32_stats['CL_r2'], M_64_32_stats['CD_r2'], M_64_32_stats['Cm_r2']),
    
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_64_32_16_8_stats['CL_rmse'], M_64_32_16_8_stats['CD_rmse'], M_64_32_16_8_stats['Cm_rmse'],
     M_64_32_16_8_stats['CL_r2'], M_64_32_16_8_stats['CD_r2'], M_64_32_16_8_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 5), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))


## Network Performance Due to Increasing Neurons

### Case 1: Model [64, 32, 16, 3]

In [22]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")

    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.003988
CD_rmse    0.005394
Cm_rmse    0.001973
CL_r2      0.999981
CD_r2      0.762696
Cm_r2      0.999539
dtype: float32


### Case 2: Model [128, 64, 32, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_128_64_32
M_128_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_128_64_32...")

    M_128_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_128_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_128_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_128_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_128_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_128_64_32_df = pd.DataFrame(M_128_64_32_metrics)

print(results_M_128_64_32_df.mean())



Run 1/20 for Model M_128_64_32...

Run 2/20 for Model M_128_64_32...

Run 3/20 for Model M_128_64_32...

Run 4/20 for Model M_128_64_32...

Run 5/20 for Model M_128_64_32...

Run 6/20 for Model M_128_64_32...

Run 7/20 for Model M_128_64_32...

Run 8/20 for Model M_128_64_32...

Run 9/20 for Model M_128_64_32...

Run 10/20 for Model M_128_64_32...

Run 11/20 for Model M_128_64_32...

Run 12/20 for Model M_128_64_32...

Run 13/20 for Model M_128_64_32...

Run 14/20 for Model M_128_64_32...

Run 15/20 for Model M_128_64_32...

Run 16/20 for Model M_128_64_32...

Run 17/20 for Model M_128_64_32...

Run 18/20 for Model M_128_64_32...

Run 19/20 for Model M_128_64_32...

Run 20/20 for Model M_128_64_32...
CL_rmse    0.003263
CD_rmse    0.004895
Cm_rmse    0.001391
CL_r2      0.999987
CD_r2      0.804653
Cm_r2      0.999772
dtype: float32


### Case 3: Model [256, 128, 64, 3]

In [24]:
# RMSE and R² metric scores on test data for Model M_256_128_64
M_256_128_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_256_128_64...")

    M_256_128_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_256_128_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_256_128_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_256_128_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_256_128_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_256_128_64_df = pd.DataFrame(M_256_128_64_metrics)

print(results_M_256_128_64_df.mean())



Run 1/20 for Model M_256_128_64...

Run 2/20 for Model M_256_128_64...

Run 3/20 for Model M_256_128_64...

Run 4/20 for Model M_256_128_64...

Run 5/20 for Model M_256_128_64...

Run 6/20 for Model M_256_128_64...

Run 7/20 for Model M_256_128_64...

Run 8/20 for Model M_256_128_64...

Run 9/20 for Model M_256_128_64...

Run 10/20 for Model M_256_128_64...

Run 11/20 for Model M_256_128_64...

Run 12/20 for Model M_256_128_64...

Run 13/20 for Model M_256_128_64...

Run 14/20 for Model M_256_128_64...

Run 15/20 for Model M_256_128_64...

Run 16/20 for Model M_256_128_64...

Run 17/20 for Model M_256_128_64...

Run 18/20 for Model M_256_128_64...

Run 19/20 for Model M_256_128_64...

Run 20/20 for Model M_256_128_64...
CL_rmse    0.002760
CD_rmse    0.004492
Cm_rmse    0.001151
CL_r2      0.999991
CD_r2      0.834945
Cm_r2      0.999841
dtype: float32


### Case 4: Model [512, 256, 128, 3]

In [25]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...

Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.002316
CD_rmse    0.004186
Cm_rmse    0.001028
CL_r2      0.999994
CD_r2      0.857025
Cm_r2      0.999874
dtype: float32


### Case 5: Model [1024, 512, 256, 3]

In [26]:
# RMSE and R² metric scores on test data for Model M_1024_512_256
M_1024_512_256_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_1024_512_256...")

    M_1024_512_256 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_1024_512_256.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_1024_512_256.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_1024_512_256.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_1024_512_256_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_1024_512_256_df = pd.DataFrame(M_1024_512_256_metrics)

print(results_M_1024_512_256_df.mean())



Run 1/20 for Model M_1024_512_256...

Run 2/20 for Model M_1024_512_256...

Run 3/20 for Model M_1024_512_256...

Run 4/20 for Model M_1024_512_256...

Run 5/20 for Model M_1024_512_256...

Run 6/20 for Model M_1024_512_256...

Run 7/20 for Model M_1024_512_256...

Run 8/20 for Model M_1024_512_256...

Run 9/20 for Model M_1024_512_256...

Run 10/20 for Model M_1024_512_256...

Run 11/20 for Model M_1024_512_256...

Run 12/20 for Model M_1024_512_256...

Run 13/20 for Model M_1024_512_256...

Run 14/20 for Model M_1024_512_256...

Run 15/20 for Model M_1024_512_256...

Run 16/20 for Model M_1024_512_256...

Run 17/20 for Model M_1024_512_256...

Run 18/20 for Model M_1024_512_256...

Run 19/20 for Model M_1024_512_256...

Run 20/20 for Model M_1024_512_256...
CL_rmse    0.002461
CD_rmse    0.004189
Cm_rmse    0.000985
CL_r2      0.999992
CD_r2      0.856510
Cm_r2      0.999883
dtype: float32


In [27]:
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_128_64_32_stats = results_M_128_64_32_df.mean()
M_256_128_64_stats = results_M_256_128_64_df.mean()
M_512_256_128_stats = results_M_512_256_128_df.mean()
M_1024_512_256_stats = results_M_1024_512_256_df.mean()

architectures = [
    '64, 32, 16, 3',
    '128, 64, 32, 3',
    '256, 128, 64, 3',
    '512, 256, 128, 3',
    '1024, 512, 256, 3'
]

table_data = [
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_128_64_32_stats['CL_rmse'], M_128_64_32_stats['CD_rmse'], M_128_64_32_stats['Cm_rmse'],
     M_128_64_32_stats['CL_r2'], M_128_64_32_stats['CD_r2'], M_128_64_32_stats['Cm_r2']),
    
    (M_256_128_64_stats['CL_rmse'], M_256_128_64_stats['CD_rmse'], M_256_128_64_stats['Cm_rmse'],
     M_256_128_64_stats['CL_r2'], M_256_128_64_stats['CD_r2'], M_256_128_64_stats['Cm_r2']),
    
    (M_512_256_128_stats['CL_rmse'], M_512_256_128_stats['CD_rmse'], M_512_256_128_stats['Cm_rmse'],
     M_512_256_128_stats['CL_r2'], M_512_256_128_stats['CD_r2'], M_512_256_128_stats['Cm_r2']),
    
    (M_1024_512_256_stats['CL_rmse'], M_1024_512_256_stats['CD_rmse'], M_1024_512_256_stats['Cm_rmse'],
     M_1024_512_256_stats['CL_r2'], M_1024_512_256_stats['CD_r2'], M_1024_512_256_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 6), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))
